<a href="https://colab.research.google.com/github/zeronek501/DLH-final/blob/master/fft_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn

In [2]:
print (torch.__version__)

1.4.0


In [3]:
a = torch.tensor([0,1,2,3], dtype=torch.float)
a = a.view(*a.shape, 1)
b = torch.zeros_like(a)
c = torch.cat([a, b], 1)
print(c)
print(torch.gather(c, 1, torch.tensor([[1,0], [1,0], [1,0], [1,0]])))
print(torch.gather(c, 1, torch.tensor([[1], [1], [1], [1]])))

tensor([[0., 0.],
        [1., 0.],
        [2., 0.],
        [3., 0.]])
tensor([[0., 0.],
        [0., 1.],
        [0., 2.],
        [0., 3.]])
tensor([[0.],
        [0.],
        [0.],
        [0.]])


In [0]:
def decompose_cmpx_tensor(a):
    return torch.split(a, 1, dim=len(a.shape)-1)

In [5]:
decompose_cmpx_tensor(c)

(tensor([[0.],
         [1.],
         [2.],
         [3.]]), tensor([[0.],
         [0.],
         [0.],
         [0.]]))

In [0]:
def cmpx_tensor_hadamard_product(a, b):
    a_r, a_i = decompose_cmpx_tensor(a)
    b_r, b_i = decompose_cmpx_tensor(b)
    c_r = a_r*b_r - a_i*b_i
    c_i = a_r*b_i + a_i*b_r
    return torch.cat([c_r,c_i], len(a.shape)-1)

In [0]:
# 1D-experiment
def test_fft_conv(in_r, w_r):
    dim = len(in_r.shape)
    if dim == 1:
        print('w_r: \n', w_r)
        w_r = torch.flip(w_r, [0])
        print('flipped w_r: \n', w_r)
        pad = nn.ConstantPad1d((1, 1), 0)
        in_r = pad(in_r)
        pad = nn.ConstantPad1d((0, 3), 0)
        w_r = pad(w_r)
    elif dim == 2:
        print('w_r: \n', w_r)
        w_r = torch.flip(w_r, [0,1])
        print('flipped w_r: \n', w_r)
        pad = nn.ConstantPad2d((1, 1, 1, 1), 0)
        in_r = pad(in_r)
        pad = nn.ConstantPad2d((0, 3, 0, 3), 0)
        w_r = pad(w_r)
    in_r = in_r.view(*in_r.shape, 1)
    w_r = w_r.view(*w_r.shape, 1)

    in_i = torch.zeros_like(in_r)
    w_i = torch.zeros_like(w_r)
    in_c = torch.cat([in_r, in_i], dim)
    w_c = torch.cat([w_r, w_i], dim)
    print('padded in_c: \n', in_c)
    print('padded w_c: \n', w_c)
    in_c_fft = torch.fft(in_c, dim)
    w_c_fft = torch.fft(w_c, dim)
    print('in_c_fft: \n', in_c_fft)
    print('w_c_fft: \n', w_c_fft)
    in_c_fft_ifft = torch.ifft(in_c_fft, dim)
    w_c_fft_ifft = torch.ifft(w_c_fft, dim)
    print('in_c_fft_ifft: \n', in_c_fft_ifft)
    print('w_c_fft_ifft: \n', w_c_fft_ifft)
    out_c_fft = cmpx_tensor_hadamard_product(in_c_fft, w_c_fft)
    print('out_c_fft: \n', out_c_fft)
    conv_c_fft = torch.ifft(out_c_fft, dim)
    print('conv_c_fft: \n', conv_c_fft)
    print('rounded conv_c_fft: \n', torch.round(conv_c_fft*1e2)/1e2)

In [0]:
in_r = torch.tensor([
    [0, 1, 2, 3], 
    [4, 5, 6, 7], 
    [8, 9, 10, 11], 
    [12, 13, 14, 15]
], dtype=torch.float)
w_r = torch.tensor([
    [0, 0, 0], 
    [0, 0, 1], 
    [0, 0, 0]
], dtype=torch.float)
test_fft_conv(in_r, w_r)

w_r: 
 tensor([[0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 0.]])
flipped w_r: 
 tensor([[0., 0., 0.],
        [1., 0., 0.],
        [0., 0., 0.]])
padded in_c: 
 tensor([[[ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.],
         [ 1.,  0.],
         [ 2.,  0.],
         [ 3.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 4.,  0.],
         [ 5.,  0.],
         [ 6.,  0.],
         [ 7.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 8.,  0.],
         [ 9.,  0.],
         [10.,  0.],
         [11.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [12.,  0.],
         [13.,  0.],
         [14.,  0.],
         [15.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.]]])
padded w_c: 
 tensor([[[0., 0.],
         [0., 0.],
         

In [0]:
in_r = torch.tensor([0, 1, 2, 3], dtype=torch.float)
w_r = torch.tensor([0, 0, 1], dtype=torch.float)
test_fft_conv(in_r, w_r)

w_r: 
 tensor([0., 0., 1.])
flipped w_r: 
 tensor([1., 0., 0.])
padded in_c: 
 tensor([[0., 0.],
        [0., 0.],
        [1., 0.],
        [2., 0.],
        [3., 0.],
        [0., 0.]])
padded w_c: 
 tensor([[1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])
in_c_fft: 
 tensor([[ 6.0000,  0.0000],
        [-4.0000,  1.7321],
        [ 0.0000, -1.7321],
        [ 2.0000,  0.0000],
        [ 0.0000,  1.7321],
        [-4.0000, -1.7321]])
w_c_fft: 
 tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])
in_c_fft_ifft: 
 tensor([[0., 0.],
        [0., 0.],
        [1., 0.],
        [2., 0.],
        [3., 0.],
        [0., 0.]])
w_c_fft_ifft: 
 tensor([[1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])
out_c_fft: 
 tensor([[ 6.0000,  0.0000],
        [-4.0000,  1.7321],
        [ 0.0000, -1.7321],
        [ 2.0000,  0.0000],
        [ 0.

In [0]:
in_r = torch.tensor([
    [0, 1, 2, 3], 
    [4, 5, 6, 7], 
    [8, 9, 10, 11], 
    [12, 13, 14, 15]
], dtype=torch.float)
w_r = torch.tensor([
    [0, 0, 0], 
    [0, 0, 1], 
    [0, 0, 0]
], dtype=torch.float)
print(w_r)
w_r = torch.flip(w_r, [0,1])
print(w_r)
pad = nn.ConstantPad2d((0, 1, 0, 1), 0)
w_r = pad(w_r)
in_r = in_r[:, :, None]
w_r = w_r[:, :, None]

in_i = torch.zeros_like(in_r)
w_i = torch.zeros_like(w_r)
in_c = torch.cat([in_r, in_i], 2)
w_c = torch.cat([w_r, w_i], 2)
print(in_c)

tensor([[0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 0.]])
tensor([[0., 0., 0.],
        [1., 0., 0.],
        [0., 0., 0.]])
tensor([[[ 0.,  0.],
         [ 1.,  0.],
         [ 2.,  0.],
         [ 3.,  0.]],

        [[ 4.,  0.],
         [ 5.,  0.],
         [ 6.,  0.],
         [ 7.,  0.]],

        [[ 8.,  0.],
         [ 9.,  0.],
         [10.,  0.],
         [11.,  0.]],

        [[12.,  0.],
         [13.,  0.],
         [14.,  0.],
         [15.,  0.]]])


In [0]:
in_c_fft = torch.fft(in_c, 2)
w_c_fft = torch.fft(w_c, 2)
print(in_c_fft)
print(w_c_fft)
print(torch.ifft(in_c_fft, 2))
print(torch.ifft(w_c_fft, 2))
out_c_fft = cmpx_tensor_hadamard_product(in_c_fft, w_c_fft)
print(out_c_fft)
conv_c_fft = torch.ifft(out_c_fft, 2)
print(conv_c_fft)

tensor([[[120.,   0.],
         [ -8.,   8.],
         [ -8.,   0.],
         [ -8.,  -8.]],

        [[-32.,  32.],
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]],

        [[-32.,   0.],
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]],

        [[-32., -32.],
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]]])
tensor([[[ 1.,  0.],
         [ 1.,  0.],
         [ 1.,  0.],
         [ 1.,  0.]],

        [[ 0., -1.],
         [ 0., -1.],
         [ 0., -1.],
         [ 0., -1.]],

        [[-1.,  0.],
         [-1.,  0.],
         [-1.,  0.],
         [-1.,  0.]],

        [[ 0.,  1.],
         [ 0.,  1.],
         [ 0.,  1.],
         [ 0.,  1.]]])
tensor([[[ 0.,  0.],
         [ 1.,  0.],
         [ 2.,  0.],
         [ 3.,  0.]],

        [[ 4.,  0.],
         [ 5.,  0.],
         [ 6.,  0.],
         [ 7.,  0.]],

        [[ 8.,  0.],
         [ 9.,  0.],
         [10.,  0.],
         [11.,  0.]],

        [[12.,  0.],
 